In [2]:
from csv import reader

In [3]:
# read data
crime_data_lines = sc.textFile('sf_data.csv')

In [4]:
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])

In [6]:
header = df_crimes.first()
header

['IncidntNum',
 'Category',
 'Descript',
 'DayOfWeek',
 'Date',
 'Time',
 'PdDistrict',
 'Resolution',
 'Address',
 'X',
 'Y',
 'Location',
 'PdId']

In [7]:
## Extract header
crimes = df_crimes.filter(lambda x: x != header)
print (crimes.take(1))
print (crimes.take(2))
print (crimes.count())

[['179005317', 'OTHER OFFENSES', 'FALSE PERSONATION TO RECEIVE MONEY OR PROPERTY', 'Tuesday', '07/11/2017', '14:32', 'SOUTHERN', 'NONE', '800 Block of BRYANT ST', '-122.40340479147905', '37.775420706711', '(37.775420706711, -122.40340479147905)', '17900531709029']]
[['179005317', 'OTHER OFFENSES', 'FALSE PERSONATION TO RECEIVE MONEY OR PROPERTY', 'Tuesday', '07/11/2017', '14:32', 'SOUTHERN', 'NONE', '800 Block of BRYANT ST', '-122.40340479147905', '37.775420706711', '(37.775420706711, -122.40340479147905)', '17900531709029'], ['179005210', 'FORGERY/COUNTERFEITING', 'MANUFACTURE OR SALE OF COUNTERFEIT GOODS', 'Tuesday', '07/11/2017', '17:27', 'SOUTHERN', 'NONE', '800 Block of BRYANT ST', '-122.40340479147905', '37.775420706711', '(37.775420706711, -122.40340479147905)', '17900521009261']]
8964


In [8]:
# Counts the number of crimes for different category.
category_id_pairs = crimes.map(lambda x: (x[1],1))
category_id_pairs.reduceByKey(lambda x,y: x+y).collect()

[('OTHER OFFENSES', 1002),
 ('FORGERY/COUNTERFEITING', 29),
 ('NON-CRIMINAL', 988),
 ('BURGLARY', 300),
 ('SEX OFFENSES, FORCIBLE', 39),
 ('ROBBERY', 187),
 ('MISSING PERSON', 263),
 ('SECONDARY CODES', 118),
 ('STOLEN PROPERTY', 66),
 ('WEAPON LAWS', 106),
 ('RECOVERED VEHICLE', 48),
 ('BRIBERY', 7),
 ('DISORDERLY CONDUCT', 18),
 ('PROSTITUTION', 20),
 ('DRUNKENNESS', 19),
 ('KIDNAPPING', 10),
 ('EXTORTION', 2),
 ('LARCENY/THEFT', 2804),
 ('VANDALISM', 650),
 ('FRAUD', 155),
 ('ASSAULT', 780),
 ('SUSPICIOUS OCC', 311),
 ('LOITERING', 4),
 ('WARRANTS', 309),
 ('VEHICLE THEFT', 353),
 ('DRUG/NARCOTIC', 167),
 ('RUNAWAY', 39),
 ('TRESPASS', 109),
 ('ARSON', 29),
 ('PORNOGRAPHY/OBSCENE MAT', 1),
 ('SEX OFFENSES, NON FORCIBLE', 1),
 ('EMBEZZLEMENT', 5),
 ('DRIVING UNDER THE INFLUENCE', 16),
 ('SUICIDE', 3),
 ('LIQUOR LAWS', 4),
 ('FAMILY OFFENSES', 1),
 ('TREA', 1)]

In [9]:
# Counts the number of crimes for different district¶
category_id_pairs = crimes.map(lambda x: (x[6],1))
category_id_pairs.reduceByKey(lambda x,y: x+y).collect()

[('SOUTHERN', 1743),
 ('TARAVAL', 660),
 ('INGLESIDE', 670),
 ('BAYVIEW', 816),
 ('PARK', 517),
 ('RICHMOND', 527),
 ('MISSION', 1190),
 ('TENDERLOIN', 535),
 ('CENTRAL', 1137),
 ('NORTHERN', 1169)]

In [13]:
# Count the number of crimes each Sunday at SF downtown
filtered_crimes = crimes.filter(lambda x: x[3]=='Sunday' and x[6]=='CENTRAL' 
                                and float(x[9])<=-122.4 and float(x[9])>=-122.5 and float(x[10])>=37.7 and float(x[10])<=37.8)
filtered_pairs = filtered_crimes.map(lambda x: (x[4],1))
results = filtered_pairs.reduceByKey(lambda x,y: x+y)
results.sortByKey().collect()

[('07/16/2017', 29), ('07/23/2017', 35), ('07/30/2017', 36)]

In [135]:
results_df = results.sortByKey().toDF()
results_renamed = results_df.selectExpr('_1 as Date','_2 as count')
results_renamed.show()

+----------+-----+
|      Date|count|
+----------+-----+
|07/16/2017|   50|
|07/23/2017|   61|
|07/30/2017|   51|
+----------+-----+



In [136]:
# date/day of week/count
filtered_crimes = crimes.filter(lambda x:x[6]=='CENTRAL')
filtered_pairs = filtered_crimes.map(lambda x: ((x[4],x[3]),1))
results = filtered_pairs.reduceByKey(lambda x,y: x+y)
results.sortByKey().collect()

[(('07/11/2017', 'Tuesday'), 42),
 (('07/12/2017', 'Wednesday'), 45),
 (('07/13/2017', 'Thursday'), 64),
 (('07/14/2017', 'Friday'), 45),
 (('07/15/2017', 'Saturday'), 57),
 (('07/16/2017', 'Sunday'), 50),
 (('07/17/2017', 'Monday'), 55),
 (('07/18/2017', 'Tuesday'), 57),
 (('07/19/2017', 'Wednesday'), 59),
 (('07/20/2017', 'Thursday'), 50),
 (('07/21/2017', 'Friday'), 58),
 (('07/22/2017', 'Saturday'), 56),
 (('07/23/2017', 'Sunday'), 61),
 (('07/24/2017', 'Monday'), 52),
 (('07/25/2017', 'Tuesday'), 51),
 (('07/26/2017', 'Wednesday'), 57),
 (('07/27/2017', 'Thursday'), 56),
 (('07/28/2017', 'Friday'), 53),
 (('07/29/2017', 'Saturday'), 68),
 (('07/30/2017', 'Sunday'), 51),
 (('07/31/2017', 'Monday'), 50)]

In [149]:
df = results.sortByKey().map(lambda x :(x[0][0],x[0][1],x[1])).toDF()
df_renamed = df.selectExpr('_1 as Date','_2 as dayOfWeek','_3 as count')
df_renamed.show(30)

+----------+---------+-----+
|      Date|dayOfWeek|count|
+----------+---------+-----+
|07/11/2017|  Tuesday|   42|
|07/12/2017|Wednesday|   45|
|07/13/2017| Thursday|   64|
|07/14/2017|   Friday|   45|
|07/15/2017| Saturday|   57|
|07/16/2017|   Sunday|   50|
|07/17/2017|   Monday|   55|
|07/18/2017|  Tuesday|   57|
|07/19/2017|Wednesday|   59|
|07/20/2017| Thursday|   50|
|07/21/2017|   Friday|   58|
|07/22/2017| Saturday|   56|
|07/23/2017|   Sunday|   61|
|07/24/2017|   Monday|   52|
|07/25/2017|  Tuesday|   51|
|07/26/2017|Wednesday|   57|
|07/27/2017| Thursday|   56|
|07/28/2017|   Friday|   53|
|07/29/2017| Saturday|   68|
|07/30/2017|   Sunday|   51|
|07/31/2017|   Monday|   50|
+----------+---------+-----+

